In [1]:
# !pip install openai

In [2]:
# !pip install --upgrade openai

In [20]:
import os
import openai
import time
import pandas as pd

In [1]:
openai.organization = ""#"X"
openai.api_key = "X"
openai.Model.list()

NameError: name 'openai' is not defined

In [22]:
template = lambda q, a: f"""<P {q} /P>. <R {a} /R>. Pensemos paso a paso."""
q = "Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras."
a = "estamal por caben 20 dados"
prompt = template(q, a)
prompt

'<P Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras. /P>. <R estamal por caben 20 dados /R>. Pensemos paso a paso.'

In [23]:
shots = [
    (template(
        "Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras.", 
        "esta mal  por caben 20 dados"), 
    "La respuesta dice 'esta mal' y ademas explica porque. Entonces, <E Coherente /E>"),
    (template(
        "María y su esposo ayer cocinaron una tortilla, la dividieron en 6 partes iguales. María comió 2/6 y su esposo 3/6. ¿Qué fracción de la tortilla quedo?", 
        "quedo un pedaso de tortilla y la ultima quedo a y la tortilla"), 
    "La respuesta tiene una fraccion en palabras. Entonces, <E Coherente /E>"),
    (template(
        "Claudio tiene 12 lápices para repartir en cantidades iguales en 3 estuches, para hacerlo, su amiga Paula le dice que deberá poner 6 lápices en cada estuche y así no le sobrará ninguno. ¿Está en lo correcto la amiga de Claudio? (Explica con tus palabras la respuesta)", 
        "paula no esta en lo correcto porque me pide en mas estuches"), 
    "La respuesta dice 'no esta en lo correco' y ademas explica porque. Entonces, <E Coherente /E> "),
    (template(
        "Francisco y Camila tienen una colección de figuras de acción de su serie favorita, Francisco tiene 4/8 de la colección y Camila tiene 2/8, si del total de figuras que Francisco y Camila tienen pierden 5/8. ¿Cuántas figuras de la colección le quedan? (Redacta adecuadamente tu respuesta)", 
        "ESTA BUENA"), 
    "La respuesta no da una representacion numericas, ni tampoco explica su respuesta. Entonces, <E Incoherente /E>"),
]
def chatgpt(message, text, temp=0.7, max_t=5):
    try: 
        output = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                  {"role": "system", "content": message},
                  {"role": "user", "content": shots[0][0]},
                  {"role": "assistant", "content": shots[0][1]},
                  {"role": "user", "content": shots[1][0]},
                  {"role": "assistant", "content": shots[1][1]},
                  {"role": "user", "content": shots[2][0]},
                  {"role": "assistant", "content": shots[2][1]},
                  {"role": "user", "content": shots[3][0]},
                  {"role": "assistant", "content": shots[3][1]},
                  {"role": "user", "content": text},
            ],
            temperature=temp,
            max_tokens=max_t,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
    except:
        print("Thinking...")
        time.sleep(1)
        return chatgpt(message, text, temp=temp, max_t=max_t)
    return output

In [24]:
%%time
system = "Usted es un clasificador de incoherencia en las respuestas a preguntas abiertas de estudiantes de matemáticas de cuarto grado. Le daré una pregunta y respuestas diferentes, y luego tendrá que decir si la respuesta es coherente o incoherente a la pregunta. Marcaremos las preguntas entre <P /P> y las respuestas entre <R /R>. Entregará la etiqueta incoherente o coherente entre <E /E>. Las respuestas incorrectas aún pueden ser coherentes, dado que la incoherencia significa que la respuesta no tiene sentido con lo que se pregunta y no necesariamente con su Correctitud."
chatgpt(system, prompt, temp=0.7, max_t=35)["choices"][0]["message"]["content"]

Wall time: 2.02 s


"La respuesta dice 'esta mal' y ademas explica porque. Entonces, <E Coherente /E>"

In [25]:
test_set = pd.read_excel("data/test_task_C1.xlsx", index_col=0)
test_set

,Q_id,Q,A,label
id,,,,
85815,4581,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...,0
64812,3535,¿Cómo creen ustedes que se reproducen las célu...,El proceso de división de una célula en dos cé...,0
84148,4491,¿ Qué frase para mejorar le darías tú a un am...,debes practicar los ejerccios,0
88565,4734,¿Qué aprendiste en la actividad del día viernes?,Nose porque no fui. A donde fueron ustedes😛😛😛😛😜,1
51688,2944,Diego compró 2 sopaipillas a $120 cada una. Pa...,las sopaipillas,1
...,...,...,...,...
54685,3109,Lorena dice que al multiplicar 3x0 le da como...,estamal poreque es 0,0
87324,4665,Alicia comió 2/9 de un chocolate. Ella dice qu...,esta mal Alicia por que la cantidad es 7/8 de...,0
73798,3946,El cuarto básico visitará el zoológico. Si son...,nose\n,1


In [26]:
q, a, label = test_set.sample(1).iloc[0]["Q A label".split()]
prompt = template(q, a)
prompt, label

('<P ¿Qué aprendiste en la actividad del día viernes? /P>. <R buena /R>. Pensemos paso a paso.',
 1)

In [27]:
chatgpt(system, prompt, temp=0.7, max_t=50)["choices"][0]["message"]["content"]

'La respuesta no responde a la pregunta, por lo que es incoherente. Entonces, <E Incoherente /E>'

In [28]:
%%time
responses = []
for i, ix in enumerate(test_set.index):
    q, a, label = test_set.loc[ix]["Q A label".split()]
    a = a.replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(system, template(q, a), temp=0.7, max_t=50)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/test_set.shape[0], ix, o, label)

0.14771048744460857 85815 La respuesta no tiene sentido con lo que se pregunta, por lo tanto es incoherente. Entonces, <E Incoherente /E> 0
0.29542097488921715 64812 La respuesta es una explicación coherente y precisa del proceso de reproducción celular. Entonces, <E Coherente /E> 0
0.4431314623338257 84148 La respuesta no proporciona una frase constructiva para mejorar a un amigo. Entonces, <E Incoherente /E> 0
0.5908419497784343 88565 La respuesta no responde a la pregunta realizada y además agrega emoticonos que no tienen relación con la consulta. Entonces, <E Incoherente /E>. 1
0.7385524372230429 51688 La respuesta no explica cuales fueron las operaciones matematicas utilizadas en el problema, ademas habla de las sopaipillas en vez de las operaciones. Entonces, <E Incoherente /E> 1
0.8862629246676514 73857 La respuesta indica que no sabe la respuesta pero no da información contradictoria con la pregunta. Entonces, <E Coherente /E> 1
1.03397341211226 68843 La respuesta dice 'no' per

In [29]:
test_c1_pred = test_set.copy()
test_c1_pred["chatgpt"] = [x[1] for x in responses]
test_c1_pred

,Q_id,Q,A,label,chatgpt
id,,,,,
85815,4581,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...,0,La respuesta no tiene sentido con lo que se pr...
64812,3535,¿Cómo creen ustedes que se reproducen las célu...,El proceso de división de una célula en dos cé...,0,La respuesta es una explicación coherente y pr...
84148,4491,¿ Qué frase para mejorar le darías tú a un am...,debes practicar los ejerccios,0,La respuesta no proporciona una frase construc...
88565,4734,¿Qué aprendiste en la actividad del día viernes?,Nose porque no fui. A donde fueron ustedes😛😛😛😛😜,1,La respuesta no responde a la pregunta realiza...
51688,2944,Diego compró 2 sopaipillas a $120 cada una. Pa...,las sopaipillas,1,La respuesta no explica cuales fueron las oper...
...,...,...,...,...,...
54685,3109,Lorena dice que al multiplicar 3x0 le da como...,estamal poreque es 0,0,La respuesta dice 'esta mal' y ademas explica ...
87324,4665,Alicia comió 2/9 de un chocolate. Ella dice qu...,esta mal Alicia por que la cantidad es 7/8 de...,0,La respuesta dice 'esta mal' y ademas explica ...
73798,3946,El cuarto básico visitará el zoológico. Si son...,nose\n,1,La respuesta no es coherente con lo que se pre...


In [30]:
test_c1_pred.to_excel("chatgpt_responses_test_c1_pred_cot_s4_v3.xlsx")

In [31]:
from sklearn.metrics import classification_report

In [32]:
print(classification_report(test_c1_pred["label"], test_c1_pred["chatgpt"].apply(lambda x: int("incoherente" in x.lower())), digits=4))

              precision    recall  f1-score   support

           0     0.9502    0.7061    0.8102       541
           1     0.4218    0.8529    0.5645       136

    accuracy                         0.7356       677
   macro avg     0.6860    0.7795    0.6873       677
weighted avg     0.8441    0.7356    0.7608       677



In [33]:
root = "D:\\github\\coherence-prediction\\"
test_label_Q = pd.read_excel(root+"gpt_label_v7_comp.xlsx").set_index("id")
test_c1_pred["label_Q"] = test_label_Q.loc[test_c1_pred.index]["tipo_preg"]
print(classification_report(test_c1_pred[test_c1_pred["label_Q"] == 3]["label"], test_c1_pred[test_c1_pred["label_Q"] == 3]["chatgpt"].apply(lambda x: int("incoherente" in x.lower())), digits=4))

              precision    recall  f1-score   support

           0     0.9746    0.8779    0.9237       262
           1     0.5224    0.8537    0.6481        41

    accuracy                         0.8746       303
   macro avg     0.7485    0.8658    0.7859       303
weighted avg     0.9134    0.8746    0.8864       303

